#Lab.04 / IBM3202 – Modelamiento Comparativo usando MODELLER

##Aspectos teóricos

Como discutimos durante nuestras conferencias, **el espacio de estructuras es bastante pequeño en comparación con el espacio de secuencias**. De hecho, clasificaciones de estructuras jerárquicas como **CATH** han demostrado que solo unas pocas proteínas del enorme conjunto de estructuras depositadas cada año en el Banco de Datos de Proteínas son conocidas por proporcionar pliegues novedosos. En este sentido, parece que hemos descubierto casi todos los pliegues de proteínas. Además, el elegante experimento realizado por Chothia & Lesk [Chothia C & Lesk AM (1986) _EMBO J_ 5(4), 823–826] reveló que, en general, **las estructuras de proteínas son bastante similares entre sí cuando su identidad de secuencia es > 30-40%**, con algunas excepciones notables a esta regla).


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/cm_01.png'/>
<figcaption>FIGURA 1. Relación entre la identidad de secuencia y la similitud estructural para proteínas globulares y de membrana

<br>Olivella M et al (2013) <i>Bioinformatics 29(13), 1589-1592</i></figcaption></center>
</figure>

Esta evidencia proporciona el marco perfecto para modelar la estructura de una proteína dada basándose en otra proteína (una plantilla) cuya estructura ha sido resuelta y que tiene una identidad de secuencia suficiente. Pero, ¿cómo funciona? Bueno, básicamente opera utilizando una gran cantidad de restricciones de distancia.

De manera similar a lo que vimos con las señales coevolutivas en nuestras conferencias, en las cuales pares de residuos se conservan independientemente de su separación en secuencia debido a su papel en estabilizar el estado nativo a través de interacciones en el espacio tridimensional, podemos agregar restricciones de interacción a una secuencia de una proteína con estructuras desconocidas basándonos en lo que observamos en la estructura de la plantilla.


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/cm_02.png'/>
<figcaption>FIGURA 2. Desde la similitud de secuencia hasta las restricciones espaciales para el modelado comparativo en MODELLER.

<br>Fiser A & Sali A (2003) <i>Methods Enzymol 374, 461-491</i></figcaption></center>
</figure>

##Generalidades

En este tutorial, perseguiremos el modelado basado en plantillas de una enzima homóloga a la PET hydrolase (PETasa) de _I. sakaiensis_ [Fecker T et al (2018) *Biophys J 114 (6), 1302-1312*] utilizando el software **MODELLER** mediante un enfoque de una sola plantilla (es decir, basado solo en una estructura de proteína). Luego, analizaremos el modelo y determinaremos su calidad utilizando un servidor en línea.


#Part 0 – Downloading and Installing the required software

Antes de comenzar, debes **recordar iniciar el entorno de ejecución alojado en Google Colab**.

Luego, debemos instalar varios programas para realizar este tutorial. Específicamente:
- **MODELLER**, un software famoso para el modelado basado en plantillas.
- **py3Dmol**, para la visualización de las estructuras de proteínas de la plantilla y modeladas.
- **biopython**, para la manipulación y recuperación de estructuras y secuencias de proteínas.

Después de realizar varias pruebas, las siguientes instrucciones de instalación son la mejor manera de configurar **Google Colab** para esta sesión de laboratorio.

1. Primero instalaremos MODELLER de la siguiente manera:

⚠️**ADVERTENCIA:** Para utilizar MODELLER, necesitarás obtener una licencia académica registrándote **[en este sitio web](https://salilab.org/modeller/registration.html)**. La clave de la licencia se enviará inmediatamente a tu dirección de correo electrónico.


In [ ]:
# Antes de ejecutar este script, asegúrate de reemplazar la Clave de Licencia de MODELLER con la que se envió después del registro en el sitio web de MODELLER.
!wget https://salilab.org/modeller/10.4/modeller-10.4.tar.gz
# Luego, extraemos la carpeta descargada que contiene MODELLER 10.1
!tar -zxf modeller-10.4.tar.gz
!echo "Extracción de MODELLER completada"
# Después, nos movemos a la carpeta de MODELLER
# con un comando automático
%cd modeller-10.4
# Y preparamos un archivo que contenga los elementos mínimos de configuración
# para la instalación, incluyendo la clave de licencia
with open('modeller_config', 'a') as f:
  f.write("2\n")
  f.write("/content/compiled/MODELLER\n")
# ¡AGREGA TU CLAVE DE LICENCIA AQUÍ!
  f.write("MODELIRANJE\n")
!./Install < modeller_config
!echo "Configuración de MODELLER completada"
%cd /content/

In [ ]:
#Creando un link simbólico
%cd modeller-10.4
!ln -sf /content/compiled/MODELLER/bin/mod10.4 /usr/bin/
%cd /content/
#Revisando si MODELLER funciona
!mod10.4 | awk 'NR==1{if($1=="usage:") print "MODELLER succesfully installed"; else if($1!="usage:") print "Something went wrong. Please install again"}'

2. Después instalamos biopython

In [ ]:
#Instalando biopython a través de pip
!pip install biopython

3. Y finalmente instalamos py3Dmol

In [ ]:
#Instalando py3Dmol usando pip
!pip install py3Dmol
#Importando el módulo instalado
import py3Dmol

#Parte	I – Obteniendo secuencias de aminoácidos de homólogos de PETasas

Normalmente, este paso comienza con una **búsqueda BLAST** (u otros tipos de búsqueda en bases de datos) de homólogos de secuencia de una proteína de interés, teniendo en cuenta los valores de e, la cobertura de secuencia y las identidades.

Aquí, omitimos la búsqueda BLAST que realizamos durante nuestro tutorial anterior y simplemente descargaremos una secuencia.

1. Descargar la secuencia con el ID de acceso BAB86909.1 utilizando Biopython, como hicimos en tutoriales anteriores:

In [ ]:
import os
from pathlib import Path
from Bio import SeqIO, Entrez

# Lista de identificadores de secuencia
seqlist = ['6ANE_A', 'BAB86909.1']

for n in seqlist:
    # Crear carpeta para nuestra secuencia
    if not os.path.exists(n):
        os.mkdir(n)
    folder = Path(n)

    # Configurar tu correo electrónico para poder usar Entrez
    Entrez.email = 'tu.correo@ejemplo.cl'

    # Aquí, configuramos un control temporal con nuestra secuencia descargada en formato fasta
    temp = Entrez.efetch(db="protein", rettype="fasta", id=n)

    # Crear un archivo fasta para escribir nuestra secuencia descargada
    aaseq_out = open(folder / "target.fasta", 'w')

    # Leer la información de la secuencia como una cadena en formato fasta
    aaseq = SeqIO.read(temp, format="fasta")

    # Escribir el registro de la secuencia en formato fasta
    SeqIO.write(aaseq, aaseq_out, "fasta")

    # Cerrar tanto el control temporal como el archivo FASTA
    temp.close()
    aaseq_out.close()

2. Si todo funcionó correctamente, tu secuencia debería haberse descargado como un archivo FASTA en una carpeta apropiada nombrada según su ID de acceso. ¿Podrías verificar cuál es la descripción y la secuencia de este archivo?

**💡 CONSEJO:** Solo obtendrás la información para la última consulta en la lista, ya que estás sobrescribiendo *aaseq*.


In [ ]:
#Obtén la descripción de la proteína
#Obtén la secuencia de la proteína


3. En este punto, se recomienda encarecidamente que cambies el nombre de la secuencia (etiquetada como ">") a algo más corto, como solo el ID o un nombre (por ejemplo, "target"). Puedes hacerlo fácilmente de forma manual abriendo este archivo en Google Colab. **Para este tutorial, de hecho, estamos cambiando el nombre a "target"**.


# Parte II – Seleccionar una estructura molde apropiada y realizar un alineamiento de secuencias para el modelado de la estructura de la proteína


Seleccionar un molde adecuado para modelar la estructura de una proteína homóloga es tan crucial como un alineamiento adecuado para posicionar correctamente los diferentes residuos.

**PREGUNTA:** ¿Qué características de una estructura cristalina crees que son importantes para elegir la mejor plantilla?

1. Como trabajaremos con la secuencia BAB86909.1, primero cambiaremos de directorio.

In [ ]:
%cd BAB86909.1

2. Encontraremos las posibles mejores moldes de toda la base de datos PDB para modelar la estructura de nuestra proteína objetivo utilizando el comando **profile.build()** de MODELLER.

   Para este propósito, necesitamos un archivo de texto que contenga una lista de secuencias no redundantes de PDB con una identidad de secuencia del 95% y un script adecuado para ejecutar MODELLER.


In [ ]:
#Descargar pdb_95.pir
!wget https://salilab.org/modeller/downloads/pdb_95.pir.gz
!gunzip pdb_95.pir.gz
#Descargar build_profile.py script desde GitHub
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/scripts/build_profile.py

In [ ]:
#Corriendo build_profile script
!mod10.4 build_profile.py
#Imprimiendo la lista de posibles moldes
!sed -n '/HITS FOUND IN ITERATION:     1/,/Weight Matrix/p;/Weight Matrix/q' build_profile.log

En este ejemplo particular, se está utilizando una matriz de similitud BLOSUM62 para determinar la identidad de secuencia entre la proteína objetivo y las posibles plantillas. Además, estamos empleando solo una iteración de búsqueda y el parámetro max_aln_evalue se establece en 0.01, lo que indica que solo se incluirán en el perfil final las secuencias con valores de e inferiores o iguales a 0.01.

Por simplicidad, simplemente imprimimos la tabla PDB desde el archivo de registro resultante generado durante este análisis.

Como puedes ver, se indican varios archivos PDB. Las columnas importantes para determinar las mejores moldes en este análisis son la quinta, sexta, séptima y octava columnas, que corresponden a la longitud de la secuencia de los hits de PDB y la proteína objetivo a modelar, su identidad de secuencia y valor de e, respectivamente.

**PREGUNTA:** A partir de este análisis, ¿cuál sería la mejor molde para modelar la estructura de nuestra secuencia objetivo?


2. Elegiremos cinco estructuras PDB en función de la identidad de secuencia y el valor de e, y seleccionaremos la plantilla más apropiada para nuestra secuencia objetivo entre ellas. Para esto, primero descargaremos estas estructuras utilizando el comando _Bio.PDB_ de Biopython, y luego utilizaremos el comando alignment.compare_structures() para evaluar la similitud estructural y de secuencia entre las posibles moldes a través del script **compare.py**.

   Por favor, tómate unos minutos para examinar el contenido de este script, en particular i) cómo se incluyen las diferentes estructuras de proteínas dentro del script; y ii) qué cadena se está utilizando de cada estructura. Esto es importante en casos donde solo una de muchas cadenas en el PDB corresponde a la proteína. Además, ten en cuenta que hay dos pasos de alineación: primero, un alineamiento de secuencias; segundo, un alineamiento estructural.


In [ ]:
#Descarga los archivos PDB usando biopython
from Bio.PDB import *
templates = ['6eqe', '7dzv', '7ec8', '7nei', '6sbn']
pdbl = PDBList()
for s in templates:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

In [ ]:
#Descarga build_profile.py script desde GitHub
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/scripts/compare.py
#Revisa el script y cambia los nombres si necesario

In [ ]:
#Ejecutando el script
!mod10.4 compare.py
#Revisando la salida
!sed -ne '/Sequence identity comparison (ID_TABLE):/,$ p' compare.log

Se pueden tomar varias decisiones basadas en los resultados de este análisis, que fueron filtrados en la celda de código anterior por simplicidad.

La comparación resultante entre las estructuras muestra que el PDB 6eqe tiene la mayor resolución (0.9 Å). En segundo lugar, 7ec8, 6sbn y 7nei forman un grupo separado de estructuras de 6eqe y 6dzv. En tercer lugar, con la excepción de la comparación entre 6eqe y 7dzv, las enzimas seleccionadas son bastante diferentes entre sí en términos de su secuencia de aminoácidos, con una diferencia promedio del 48% (el porcentaje de diferencia de secuencia corresponde a los números en el lado derecho del gráfico).

Así, considerando la identidad de secuencia entre la secuencia objetivo y 6eqe, su alta resolución y otras características de calidad cristalográfica (que puedes verificar en el **[sitio web del PDB](https://www.rcsb.org/)**), esta es la mejor candidata para el modelado de una sola plantilla.


3. Ahora, **alinearemos la secuencia de nuestra proteína de molde con la secuencia de nuestra proteína objetivo**, para que podamos modelar la estructura.

   ¿Qué tan difícil es? ¡En absoluto! Simplemente descarga el script **align2d.py** en tu carpeta de trabajo, verifica el script para comprobar cómo se evocan las secuencias del objetivo y la estructura de la proteína, y ejecuta el script como lo hemos hecho antes.


In [ ]:
#Descargando align2D.py script desde GitHub
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/scripts/align2D.py
#Revisa este script y cambia los nombres de ser necesario

In [ ]:
#Ejecutando el align2D script
!mod10.4 align2D.py

4. Terminarás con dos archivos nuevos (aligned.ali y aligned.fasta) que contienen el alineamiento par a par de las secuencias del objetivo y de la plantilla. Carga el archivo FASTA en [Alignment Viewer 2.0](https://fast.alignmentviewer.org/). También puedes utilizar nuestro visor de MSA montado en Colab a continuación:


In [ ]:
#@title Protein MSA Viewer en Google Colab
#The following code is modified from the wonderful viewer developed by Damien Farrell
#https://dmnfarrell.github.io/bioinformatics/bokeh-sequence-aligner

#Importing all modules first
import os, io, random
import string
import numpy as np

from Bio.Seq import Seq
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

import panel as pn
import panel.widgets as pnw
pn.extension()

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, Grid, Range1d
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot

#Setting up the amino color code according to Zappo color scheme
def get_colors(seqs):
    #make colors for bases in sequence
    text = [i for s in list(seqs) for i in s]
    #Use Zappo color scheme
    clrs =  {'K':'red',
             'R':'red',
             'H':'red',
             'D':'green',
             'E':'green',
             'Q':'blue',
             'N':'blue',
             'S':'blue',
             'T':'blue',
             'A':'blue',
             'I':'blue',
             'L':'blue',
             'M':'blue',
             'V':'blue',
             'F':'orange',
             'Y':'orange',
             'W':'orange',
             'C':'blue',
             'P':'yellow',
             'G':'orange',
             '-':'white'}
    colors = [clrs[i] for i in text]
    return colors

#Setting up the MSA viewer
def view_alignment(aln, fontsize="9pt", plot_width=800):
    """Bokeh sequence alignment view"""

    #make sequence and id lists from the aln object
    seqs = [rec.seq for rec in (aln)]
    ids = [rec.id for rec in aln]
    text = [i for s in list(seqs) for i in s]
    colors = get_colors(seqs)
    N = len(seqs[0])
    S = len(seqs)
    width = .4

    x = np.arange(1,N+1)
    y = np.arange(0,S,1)
    #creates a 2D grid of coords from the 1D arrays
    xx, yy = np.meshgrid(x, y)
    #flattens the arrays
    gx = xx.ravel()
    gy = yy.flatten()
    #use recty for rect coords with an offset
    recty = gy+.5
    h= 1/S
    #now we can create the ColumnDataSource with all the arrays
    source = ColumnDataSource(dict(x=gx, y=gy, recty=recty, text=text, colors=colors))
    plot_height = len(seqs)*15+50
    x_range = Range1d(0,N+1, bounds='auto')
    if N>100:
        viewlen=100
    else:
        viewlen=N
    #view_range is for the close up view
    view_range = (0,viewlen)
    tools="xpan, xwheel_zoom, reset, save"

    #entire sequence view (no text, with zoom)
    p = figure(title=None, width= plot_width, height=50,
               x_range=x_range, y_range=(0,S), tools=tools,
               min_border=0, toolbar_location='below')
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                 line_color=None, fill_alpha=0.6)
    p.add_glyph(source, rects)
    p.yaxis.visible = False
    p.grid.visible = False

    #sequence text view with ability to scroll along x axis
    p1 = figure(title=None, width=plot_width, height=plot_height,
                x_range=view_range, y_range=ids, tools="xpan,reset",
                min_border=0, toolbar_location='below')#, lod_factor=1)
    glyph = Text(x="x", y="y", text="text", text_align='center',text_color="black",
                text_font="monospace",text_font_size=fontsize)
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                line_color=None, fill_alpha=0.4)
    p1.add_glyph(source, glyph)
    p1.add_glyph(source, rects)

    p1.grid.visible = False
    p1.xaxis.major_label_text_font_style = "bold"
    p1.yaxis.minor_tick_line_width = 0
    p1.yaxis.major_tick_line_width = 0

    p = gridplot([[p],[p1]], toolbar_location='below')
    return p

#Loading the viewer by indicating the MSA file and format to read
#@markdown Nombre del archivo MSA (y su extensión)
MSAfile = 'aligned.fasta' #@param {type:"string"}
MSAformat = 'fasta' #@param {type:"string"}
aln = AlignIO.read(MSAfile,MSAformat)
p = view_alignment(aln, plot_width=900)
pn.pane.Bokeh(p)

Verás que algo extraño está sucediendo: **¡un segmento grande del extremo N-terminal no tiene residuos equivalentes en la estructura de la plantilla!**


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/cm_03.png'/>
</center>
</figure>

Este ejercicio lleva a una recomendación general: siempre debes verificar toda la información posible sobre tus secuencias y estructuras biológicas para identificar si estos conflictos tienen alguna explicación biológica. **ESTO ES FUNDAMENTAL**.


5. Ve al sitio web del PDB y verifica la anotación de la secuencia de la estructura que estás utilizando como molde (en este caso, 6EQE) en **Protein Feature View**. ¿Qué ves?

   Con base en tus observaciones, decide si debes truncar la secuencia de tu objetivo antes de modelar, editar tu secuencia objetivo y repetir el alineamiento, si es necesario.


# Parte III - Generar y visualizar un modelo comparativo utilizando MODELLER


1. Una vez alineadas tus secuencias objetivo y de molde, utiliza el script **model-single.py** para obtener finalmente una estructura de tu objetivo mediante el modelado comparativo. Nuevamente, lee el script y verifica cómo se llaman las secuencias y estructuras en MODELLER a través de estos scripts. En este caso, también estamos realizando este paso en una carpeta separada.

   Ten en cuenta que un modelo no es suficiente, ya que hay una función de energía que define la posición óptima de las coordenadas atómicas, por lo que diferentes modelos tendrán diferentes energías. En general, se generan entre 50 y 100 para una evaluación suficiente.

**💡 CONSEJO:** Para nuestro ejemplo, la generación de 50 modelos tarda alrededor de 15 minutos en Google Colab, mientras que 10 modelos se generan en aproximadamente 3 minutos. Puedes editar el número de modelos a generar en el script `model-single.py`.


In [ ]:
#Crea una carpeta nueva y copia archivos desde MODELLER
%mkdir model-single
%cd model-single
%cp ../6eqe.pdb .
%cp ../aligned.ali .

In [ ]:
#Descarga el model-single.py script desde GitHub
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/scripts/model-single.py
#Revisa el script y cambia los nombres de ser necesario

In [ ]:
#Ejecuta model-single script
!mod10.4 model-single.py

2. El resultado de este proceso es un conjunto de archivos PDB, cada uno de ellos correspondiente a un modelo comparativo de nuestra proteína objetivo, numerados del 1 hasta el número total de modelos solicitados durante el modelado comparativo.

   Además, la salida **model-single.log** tiene la energía potencial total para cada estructura, según la puntuación DOPE (discrete optimized protein energy) de MODELLER. Por simplicidad, este script se modificó para indicar el modelo con la mejor puntuación DOPE. Trabajaremos solo con el modelo con la mejor puntuación durante el resto de la sesión.

   Como ejemplo, nuestro mejor modelo durante la preparación de este tutorial mostró la siguiente puntuación DOPE:


```
Top model: target.B99990025.pdb (DOPE score -28735.180)
```

3. Antes de verificar la calidad de nuestro modelo, echaremos un vistazo en **py3Dmol**.

**💡 CONSEJO:** Estamos creando una copia de nuestro modelo y cambiando el identificador de cadena de A a B, para cargar ambas estructuras en py3Dmol.


In [ ]:
# Copiando nuestro mejor modelo con un nuevo identificador de cadena
!sed "s/ A / B /g" target.B99990004.pdb > bestmodel.pdb

# Configurando py3Dmol para la visualización
view = py3Dmol.view()
# Cargando la plantilla
view.addModel(open('6eqe.pdb', 'r').read(), 'pdb')
# Cargando el modelo con la mejor puntuación DOPE
view.addModel(open('bestmodel.pdb', 'r').read(), 'pdb')
# Coloreando las estructuras por el identificador de cadena
view.setStyle({'cartoon': {'colorscheme': 'chain'}})
view.zoomTo()
view.setBackgroundColor('white')
view.show()

4. Finally, to check the stereochemical quality of the model and its comparison to experimentally solved structures, we will use the [SAVES server](https://saves.mbi.ucla.edu), which employs several structure-based scoring strategies:

* **VERIFY3D** (i.e. compatibility of an atomic 3D model to its 1D sequence when compared tothe energetics of good structures from the PDB).
* **ERRAT** (i.e. quality of non-bonded interactions of a region when compared to similar regions from highly refined structures).
* **PROCHECK** (stereochemical and geometrical quality of the model, via Ramachandran plots, sidechain rotamers, etc).

4. Finalmente, para verificar la calidad estereoquímica del modelo y su comparación con estructuras resueltas experimentalmente, utilizaremos el [servidor SAVES](https://saves.mbi.ucla.edu), que emplea varias estrategias de puntuación basadas en la estructura:

   * **VERIFY3D** (es decir, compatibilidad de un modelo 3D atómico con su secuencia 1D en comparación con la energética de estructuras buenas del PDB).
   * **ERRAT** (es decir, calidad de las interacciones no enlazadas de una región en comparación con regiones similares de estructuras altamente refinadas).
   * **PROCHECK** (calidad estereoquímica y geométrica del modelo, a través de gráficos de Ramachandran, rotámeros de cadenas laterales, etc).


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/cm_04.png'/>
</center>
</figure>

Si bien discutiremos algunos de estos resultados al final de este tutorial, te animamos a leer [este artículo](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1007449), que contiene más recomendaciones sobre el modelado comparativo de estructuras de proteínas.


**¡Esto concluye el cuarto tutorial! ¡Buena ciencia!**
